In [2]:
import numpy as np
import pandas as pd

from typing import Optional
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast, GPT2LMHeadModel
import fastai
from fastai.text.all import *
import re

c:\Users\kimso\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 노래 데이터

In [ ]:
lyrics1 = pd.read_csv("./datasets/label_result_song_short.csv")

In [ ]:
lyrics2 = pd.read_csv("./datasets/translated_lyrics.csv")

,index,id,title,singer,genres,lyrics,translated_lyrics
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,Ay ay ay ay ay
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,거울 속에 내 표정 봐 봐
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,느낌 그래서 좋아요. 기다려온 D-하루
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,연습했던 손든 인사도 그대로 하면 돼
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,머리스타일 확인하고 한 번 회전 주변


In [ ]:
lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

### 노래 데이터 최종

In [ ]:
# 각 id에 대해 sentiment_label의 빈도를 계산하여, 가장 많이 나온 2개의 감정을 새로운 칼럼으로 추가

def get_top_two_sentiments(sentiments):
    sentiment_counts = sentiments.value_counts()
    # 가장 많은 2개 감정을 반환
    top_two = sentiment_counts.head(2).index.tolist()
    # 2개 미만일 경우 빈 값 처리
    return top_two + [None] * (2 - len(top_two))

# groupby와 agg를 활용해 변환
lyrics_final = lyrics.groupby("id").agg({
    "title": "first",   # 첫 번째 값 유지
    "singer": "first",  # 첫 번째 값 유지
    "genres": "first",  # 첫 번째 값 유지
    "lyrics": "first",  # 첫 번째 값 유지
    "translated_lyrics": list,  # 리스트로 묶음
    "sentiment_label": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment_label에서 두 개의 감정을 각각의 칼럼으로 분리
lyrics_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(lyrics_final['sentiment_label'].tolist(), index=lyrics_final.index)

# 불필요한 sentiment_label 열 삭제
lyrics_final = lyrics_final.drop(columns=["sentiment_label"])

In [9]:
lyrics_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 755 non-null    int64 
 1   title              755 non-null    object
 2   singer             755 non-null    object
 3   genres             755 non-null    object
 4   lyrics             755 non-null    object
 5   translated_lyrics  755 non-null    object
 6   top_sentiment      755 non-null    object
 7   second_sentiment   755 non-null    object
dtypes: int64(1), object(7)
memory usage: 47.3+ KB


## 소설 데이터

In [ ]:
novel = pd.read_csv("../0206/results/classified_novel1.csv")
novel.head()

In [8]:
len(novel[novel["file_id"] == "WARW1800000007"])

9543

- 소설의 길이때문에 오류나나.. 해서 100개씩 묶어서 그룹화

In [ ]:
# 같은 title을 가진 데이터에서 100개씩 묶어 그룹화
novel["group"] = novel.groupby("title").cumcount() // 100  # 100개 단위 그룹 생성
novel.head()

In [ ]:
novel_final = novel.groupby(["file_id", "group"]).agg({
    "title": "first",  # 대표 file_id
    "text": list,  # 100개씩 묶음
    "sentiment": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment에서 두 개의 감정을 각각의 칼럼으로 분리
novel_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(novel_final['sentiment'].tolist(), index=novel_final.index)

# 불필요한 sentiment 칼럼 삭제
novel_final = novel_final.drop(columns=["sentiment", "group", "file_id"])

novel_final.head()

In [11]:
novel_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8972 entries, 0 to 8971
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             8972 non-null   object
 1   text              8972 non-null   object
 2   top_sentiment     8972 non-null   object
 3   second_sentiment  8963 non-null   object
dtypes: object(4)
memory usage: 280.5+ KB


In [ ]:
## 두 번째 감정이 없으면 첫 번재랑 같게.
novel_final["second_sentiment"] = novel_final["second_sentiment"].fillna(novel_final["top_sentiment"])

In [13]:
novel_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8972 entries, 0 to 8971
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             8972 non-null   object
 1   text              8972 non-null   object
 2   top_sentiment     8972 non-null   object
 3   second_sentiment  8972 non-null   object
dtypes: object(4)
memory usage: 280.5+ KB


In [14]:
novel_final.to_csv("novel_final.csv")

## KoGPT2

### 사전 테스트

In [18]:
## 이제부턴 이걸로.
#lyrics_final = pd.read_csv("lyrics_final.csv")         이상하게 이렇게 하면 가사 부분 합치는 대서 결과가 달라짐,, 
novel_final = pd.read_csv("novel_final.csv")

In [19]:
# KoGPT2 기본 모델 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [20]:
novel_final["top_sentiment"].unique()

array(['분노', '상처', '당황', '불안', '기쁨', '슬픔'], dtype=object)

In [21]:
# 감정 및 노래 가사 설정
top_sentiment = "슬픔"
second_sentiment = "기쁨"
lyrics = " ".join(lyrics_final["translated_lyrics"][0])
lyrics

'햇살은 우릴 위해 내리고  바람도 서롤 감싸게 했죠  우리 웃음속에 계절은 오고 또 갔죠 바람에 흔들리는 머릿결  내게 불어오는 그대 향기  예쁜 두 눈도 웃음 소리도  모두가 내 것이었죠 이런 사랑 이런 행복 쉽다 했었죠 이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서 그댄 나를 떠나간다 해도 난 그댈 보낸 적 없죠 여전히 그댄 나를 살게 하는  이율테니  이런 사랑 이런 행복 쉽다 했었죠  이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서  그댄 나를 떠나간다 해도  난 그댈 보낸 적 없죠 기다림으로 다시 시작일테니  얼마나 사랑했는지 얼마나 또 울었는지 그대여 한순간 조차 잊지 말아요  거기 떠나간 그 곳에서 날 기억하며 기다려요 하루씩 그대에게 다가가는 나 일테니'

In [22]:
# 입력 텍스트 구성
input_text = f"{top_sentiment} | {second_sentiment} | {lyrics}"
input_text

'슬픔 | 기쁨 | 햇살은 우릴 위해 내리고  바람도 서롤 감싸게 했죠  우리 웃음속에 계절은 오고 또 갔죠 바람에 흔들리는 머릿결  내게 불어오는 그대 향기  예쁜 두 눈도 웃음 소리도  모두가 내 것이었죠 이런 사랑 이런 행복 쉽다 했었죠 이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서 그댄 나를 떠나간다 해도 난 그댈 보낸 적 없죠 여전히 그댄 나를 살게 하는  이율테니  이런 사랑 이런 행복 쉽다 했었죠  이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서  그댄 나를 떠나간다 해도  난 그댈 보낸 적 없죠 기다림으로 다시 시작일테니  얼마나 사랑했는지 얼마나 또 울었는지 그대여 한순간 조차 잊지 말아요  거기 떠나간 그 곳에서 날 기억하며 기다려요 하루씩 그대에게 다가가는 나 일테니'

In [23]:
# 토큰화
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 문장 생성
gen_ids = model.generate(
    input_ids,
    max_length=500,  # 최대 생성 길이 설정
    repetition_penalty=2.0,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True
)

# 결과 디코딩
generated_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
print(generated_text)

슬픔 | 기쁨 | 햇살은 우릴 위해 내리고  바람도 서롤 감싸게 했죠  우리 웃음속에 계절은 오고 또 갔죠 바람에 흔들리는 머릿결  내게 불어오는 그대 향기  예쁜 두 눈도 웃음 소리도  모두가 내 것이었죠 이런 사랑 이런 행복 쉽다 했었죠 이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서 그댄 나를 떠나간다 해도 난 그댈 보낸 적 없죠 여전히 그댄 나를 살게 하는  이율테니  이런 사랑 이런 행복 쉽다 했었죠  이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서  그댄 나를 떠나간다 해도  난 그댈 보낸 적 없죠 기다림으로 다시 시작일테니  얼마나 사랑했는지 얼마나 또 울었는지 그대여 한순간 조차 잊지 말아요  거기 떠나간 그 곳에서 날 기억하며 기다려요 하루씩 그대에게 다가가는 나 일테니 언제나 너를 그리워해요
그대 곁에 있어줘서 고마워요, 그리고 이제부터라도 널 만나고 싶어서 더 이상 미안한 마음 없어요.
사랑하는 나의 모든 것을 다 안고 가겠어요,
그리고 지금껏 당신을 기다리고 있는 당신의 모습을 영원히 잊을 거예요. 오늘 아침신문 보시죠.
먼저 조선일보입니다.
오늘 오전 10시부터 서울 광화문광장에서 '박근혜 퇴진 비상국민행동'이 주최로 열리는 집회가 열렸습니다.
집회 참가자들은 박 대통령의 즉각적인 하야와 국정 정상화를 촉구하고 있습니다.
이들은 "대통령의 사과와 책임자 처벌, 대통령 권한대행 체제 전환" 등을 요구하면서 청문회와 특검 도입 등 검찰 개혁과 국정조사를 요구하고 나섰는데 특히 새누리당 비주류 의원들이 탄핵에 동참할 움직임을 보이고 있다고 전했는데.
새정치민주연합 문재인 대표는 어제 최고위원회의에서 "이번 주말 촛불 민심은 새로운 대한민국을 향한 

- 사전학습된 자료가 뉴스라든가 그런거라서 이렇게 이상하게 나온다..

### Fine-Tuning

https://velog.io/@yeah7598/KoGPT2-%EB%8F%99%ED%99%94-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0

In [10]:
## 이제부턴 이걸로.
novel_final = pd.read_csv("novel_final.csv")

- 여기 참고해서 이어서 ㄱㄱ

In [12]:
data = novel_final.copy()

In [13]:
data.columns

Index(['Unnamed: 0', 'title', 'text', 'top_sentiment', 'second_sentiment'], dtype='object')

- 감정 : '상처', '불안', '기쁨', '슬픔', '분노', '당황'

In [14]:
from sklearn.preprocessing import LabelEncoder

# 감정 카테고리
emotions = ['상처', '불안', '기쁨', '슬픔', '분노', '당황']

# LabelEncoder 초기화
le = LabelEncoder()
le.fit(emotions)

LabelEncoder()

In [15]:
# 감정1과 감정2를 숫자로 변환
data['감정1_encoded'] = le.transform(data['top_sentiment'])
data['감정2_encoded'] = le.transform(data['second_sentiment'])

# 수치형 감정1, 감정2와 텍스트 결합
data['input_text'] = data['감정1_encoded'].astype(str) + " | " + data['감정2_encoded'].astype(str) + " | " + data['text']

data['input_text'].head()

0    2 | 1 | ['01범보다 무서운 곶감', '화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로서 가장 쉽게 기억해낸 이야기인 셈이다. 설화 앞뒤에 교훈적 해석을 덧붙이고 있음은 화자의 습관화된 태도의 한 모습이기도 하다. 어려서 조모로부터 들었다고 했다.', '그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 뭐이냐 하먼 자연~간 제절루 되야지 어거지루는 못 살어, 사람이.', '그래 옛날, 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성(무서운) 거여.', '애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가, 응? 그 집 문앜이 와 섰어 지금. 옛날이는 호랭이가 말두 하구 그랴. 그래 인제 그 할머니가 애기를 달램성 왼갖 소리를 다 햐. ‘호랭이 왔다’구 해두 울구우, ‘괭이 왔다’두 울구, 왼갖 소리를 다 해두 울어.', '그랑깨 꼭감을,', '“아나, 꼭감.”', '“아가, 꼭감.”', '그랑깨 꺄. 안 울어.', '호랭이 생각이? ‘야아, 이거 내가 젤 무선 짐승인디 말여? 나보다 무서웅 게 익구나. 호랭이, 꼭감이, 꼭갬이 참 무석구나....
1    4 | 2 | ['<봄꿩은 제 울음에 저 죽는다>', '그 말과 같아서 사램이 잘못 되머넌 하는 말여.', '지금 강 강사가 말여. 뭘 잘못 돼서 다치게 되머넌…', '아 그, ‘안 했이먼 몰를 긴디 지가 했잉깨 그렇지…’', '그래,', '<봄꿩은 제 울음에 죽는다>', '지가 해놓고 죽는다 그 말이지.', '그래 봄 꿩은 지가 워디 묻혔능가 모르는디 “껄 껄” 항깨루 거깄는 줄 알구 (사람이) 찾아 가능 기여.', '07뱀을 밴 여자', '화자의 체험담이다. 정상적인 임신이 아니고 잘못하여 사람이 생물을 밴 것을 이 지역 사람들은 흔히 ‘농알(어떤 이는 ‘용알’이라고도 함. 의학상으로는 포도상 가태)뱄다’고 말한다. 임신한 여자가 배가 아프다고 하기에 가만히 진단해보니 농알을 밴 듯했다

In [16]:
from transformers import PreTrainedTokenizerFast, AutoModelWithLMHead

# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
c:\Users\kimso\anaconda3\envs\test\lib\site-packages\transformers\models\auto\modeling_auto.py:1881: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [17]:
# 데이터셋 클래스 정의
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encodes(self, x): 
        # Tokenizing input text with padding and truncation
        encodings = self.tokenizer(x, truncation=True, padding='max_length', max_length=256, return_tensors="pt")
        
        # Extracting input_ids and attention_mask
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        
        print(f"Tokenized: {input_ids.shape} - {input_ids}")
        return input_ids.squeeze(0), attention_mask.squeeze(0)

    def decodes(self, x):
        return self.tokenizer.decode(x, skip_special_tokens=True)


In [ ]:
## 용량? 이슈로 잠시 조정..
data = data[:int(len(data)*0.1)]

In [19]:
# gpu 사용
device = torch.device("cuda")

In [20]:
print(device)

cuda


In [22]:
# 텍스트 데이터를 리스트 형태로 변환
train = data['input_text'][:int(len(data)*0.9)].tolist()
test = data['input_text'][int(len(data)*0.9):].tolist()

splits = [[0],[1]]

# DataLoader 초기화
tls = TfmdLists([train,test], TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
batch,seq_len = 4, 64
dls = tls.dataloaders(bs=batch, seq_len=seq_len, num_workers=4, pin_memory=True)
dls.device = device

Tokenized: torch.Size([8074, 256]) - tensor([[ 9045,   739,   466,  ..., 10912,   387, 12187],
        [ 9130,   739,   466,  ...,  9338,  9236,  7756],
        [ 9130,   739,   466,  ...,  6933, 19202, 18215],
        ...,
        [ 9085,   739,   466,  ..., 10360, 24391, 26049],
        [ 9085,   739,   466,  ...,   377, 12187,  9194],
        [ 9085,   739,   466,  ..., 12462, 32574, 35698]])
Tokenized: torch.Size([8074, 256]) - tensor([[ 9045,   739,   466,  ..., 10912,   387, 12187],
        [ 9130,   739,   466,  ...,  9338,  9236,  7756],
        [ 9130,   739,   466,  ...,  6933, 19202, 18215],
        ...,
        [ 9085,   739,   466,  ..., 10360, 24391, 26049],
        [ 9085,   739,   466,  ...,   377, 12187,  9194],
        [ 9085,   739,   466,  ..., 12462, 32574, 35698]])
Tokenized: torch.Size([898, 256]) - tensor([[ 9085,   739,   466,  ...,  9047, 30425, 10058],
        [ 9085,   739,   466,  ...,   377, 12187,  9194],
        [ 9085,   739,   466,  ...,  8707,  9106, 

In [27]:
from fastai.text.all import Learner, CrossEntropyLossFlat, Perplexity, Callback

# DataLoader와 모델 확인 후, Learner 초기화
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=Perplexity())

In [28]:
print(type(learn))

<class 'fastai.learner.Learner'>


In [ ]:
learn.fine_tune(1)

In [ ]:
# import torch

# print("PyTorch Version:", torch.__version__)
# print("CUDA Available:", torch.cuda.is_available())
# print("CUDA Device Count:", torch.cuda.device_count())
# print("Current Device:", torch.cuda.current_device() if torch.cuda.is_available() else "None")
# print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

PyTorch Version: 2.4.1
CUDA Available: True
CUDA Device Count: 1
Current Device: 0
Device Name: NVIDIA GeForce RTX 3060 Ti


- test

In [ ]:
# 감정1과 감정2를 숫자로 변환
# LabelEncoder 초기화
le = LabelEncoder()
le.fit(emotions)

lyrics_final['감정1_encoded'] = le.transform(lyrics_final['top_sentiment'])
lyrics_final['감정2_encoded'] = le.transform(lyrics_final['second_sentiment'])


In [ ]:
lyrics_final['감정1_encoded'][3]

In [ ]:
## 숫자 고르기
i = 3

#final = lyrics_final[3]

lyrics = " ".join(lyrics_final["translated_lyrics"][i])

# 'translated_lyrics' 컬럼의 각 리스트 요소를 하나의 문자열로 합침
prompt = lyrics_final['감정1_encoded'][i].astype(str) + " | " + lyrics_final['감정2_encoded'][i].astype(str) + " | " + lyrics
prompt

In [ ]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = learn.model.generate(inp,
                           max_length=256,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 
tokenizer.decode(preds[0].cpu().numpy())